# Accessing maps and template data

In [ ]:
import siibra
from nilearn import plotting

## Semantic concepts, spatial objects, and image data

The main concepts in `siibra` are modelled in three levels: Semantic concepts, spatial concepts, and concrete (often image) data.

#### 1. Semantic concepts: Atlases, reference spaces and brain parcellations

On the top level, we have classes modeling the semantic concepts: atlases, reference spaces, brain parcellations. Objects of these classes do not include actual image or other data.

In [ ]:
mni152space = siibra.spaces.MNI152_2009C_NONL_ASYM
# works as well: siibra.spaces['mni152']
julichbrain = siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_MAPS_2_9
# works as well: siibra.parcellations['julich']
type(mni152space),type(julichbrain)

Spaces and parcellations typically refer to entities in the EBRAINS knowledge graph, modelled in the MINDS and openMINDS standards. `siibra` stores their identifiers.

In [ ]:
print(mni152space.id)

Parcellation objects define hierarchical regiontrees, and allow to search brain regions.

In [ ]:
julichbrain.find_regions('lateral occipital')

#### 2. Spatial concepts: reference templates, parcellation maps

Reference spaces and parcellations have spatial representations in specific coordinate systems. For example, a space provides a reference template, and a parcellation provides parcellation maps of possibly different type in different reference spaces. Spatial objects are obtained from semantic objects by specifying a space.

In [ ]:
jub_maxpm = julichbrain.get_map(mni152space, maptype=siibra.MapType.LABELLED)
# works as well: jub_maxpm = julichbrain.get_map("mni152",maptype="labelled")
jub_pmaps = julichbrain.get_map(mni152space, maptype=siibra.MapType.CONTINUOUS)
type(jub_maxpm), type(jub_pmaps)

#### 3. Image data: Template images, labelled maps, probabilistic maps, volumes of interest

`siibra` follows a lazy scheme of loading image data, because some of the unerlying data is large. To get access to actual image data, we apply the `fetch` method of spatial objects, which allows optional specification of resolution and regions of interest.

In [ ]:
jubrain_cmap = julichbrain.get_colormap()
plotting.plot_roi(jub_maxpm.fetch(), cmap=jubrain_cmap)

The same for the Colin27 space:

In [ ]:
colin = siibra.spaces['colin'].get_template().fetch()
colin_mpm = siibra.parcellations['julich'].get_map('colin').fetch()
plotting.plot_roi(colin_mpm,bg_img=colin, cmap=jubrain_cmap)

Of course, we can also retrieve BigBrain template and maps:

In [ ]:
bigbrainspace = siibra.spaces.BIG_BRAIN
bigbraintemplate = bigbrainspace.get_template()

layerparcellation = siibra.parcellations['cortical layers']
layermap = layerparcellation.get_map(bigbrainspace)

# for Bigbrain, we select a coarser resolution, it's too large otherwise
img = layermap.fetch(resolution_mm=0.64)
tpl = bigbraintemplate.fetch(resolution_mm=0.64)

plotting.view_img(img,bg_img=tpl,opacity=.4,symmetric_cmap=False)

#### 4. Surface meshes

`siibra` also provides access to surfaces. A popular reference space is the freesurfer fsaverage surface space, which comes in three variants: white matter surface, pial surface, and inflated surface.

In [ ]:
surfmap = julichbrain.get_map('fsaverage').fetch(variant="inflated")
plotting.view_surf(
    [surfmap['vertices'], surfmap['faces']], 
    surfmap['labels'], 
    cmap=jubrain_cmap, symmetric_cmap=False, colorbar=False)

## Using an atlas to simplify access to spaces and parcellations

The `siibra` atlas class facilitates the work with parcellations and spaces by bringing them into context. The above can be performed in short form:

In [ ]:
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS
template_img = atlas.get_template("bigbrain").fetch(resolution_mm=0.64)
layer_img = atlas.get_map(space="bigbrain",parcellation="layers").fetch(resolution_mm=0.64)
plotting.view_img(layer_img,bg_img=template_img,opacity=.5,symmetric_cmap=False)

The atlas object allows to select different parcellations.

In [ ]:
julichbrain_mpm_left = atlas.get_map("mni152","julich-brain").fetch()
plotting.plot_roi(julichbrain_mpm_left, cmap=jubrain_cmap)
# Note: using fetch_all(), we would have obtained both hemispheres
    
bundles_mpm = atlas.get_map("mni152","long bundles").fetch()
plotting.plot_roi(bundles_mpm)

A similar logic applies to select regions and build region masks.

In [ ]:
# retrieve a binary mask of the selected region(s)
v1l= atlas.get_region('v1 left')
v1_mask_l = v1l.build_mask("mni152")
plotting.plot_roi(v1_mask_l)

# retrieve a continuous regional map of the selected region, if available
v1_pmap_l = v1l.get_regional_map("mni152","continuous")
plotting.plot_roi(v1_pmap_l.fetch())

## Extracting volumes of interest from high-resolution data

To access BigBrain Data at higher resolution, we specify a rectangular volume of interest spanned by two 3D points in physical coordinates.

In [ ]:
minpoint = (-3.979, -61.256, 3.906)
maxpt = (5.863, -55.356, -2.487)

The volume of interest definition is requested from the desired space. Here, we use the BigBrain space of course.

In [ ]:
voi = atlas.get_voi('bigbrain',minpoint,maxpt)

Now we can extract a chunk from the BigBrain template a full resolution of 20 micron using this volume of interest.

In [ ]:
minpoint = (-3.979, -61.256, 3.906)
maxpt = (5.863, -55.356, -2.487)
voi = atlas.get_voi('bigbrain',minpoint,maxpt)
bigbrain = atlas.get_template('bigbrain')
bigbrainchunk = bigbrain.fetch(resolution_mm=0.02, voi=voi)
plotting.view_img(bigbrainchunk,None,cmap='gray')

The resulting image chunk sits properly in its reference space, so we can also plot it in anatomical context of the low-resolution whole brain model.

In [ ]:
# Note that we already fetched the template volume above:
# tpl=bigbraintemplate.fetch(resolution_mm=0.64)
plotting.plot_roi(bigbrainchunk,bg_img=tpl)

We can apply this volume of interest to extract chunks from other objects in the same space, like parcellation maps. Here we use the coritcal layer maps of BigBrain. We can use the LabelledParcellation object for the cortical layer maps that we requested further above, but no call its `fetch()` method again with a different resolution and the volume of interest specification.

In [ ]:
mask = layermap.fetch(resolution_mm=0.08,voi=voi)
plotting.view_img(mask,bg_img=bigbrainchunk,opacity=.5,symmetric_cmap=False)